## AUC-ML: ID3 algorithm 

## Introduction

In this section, we are going to build a decision tree using the ID3 algorithm. This is a greedy algorithm, which produces the most optimal decision tree based on the information gain at each split. Remember the formulas for entropy and information gain:

$$Entropy =-\sum_{i=0}^n p_i(x)log(p(x))$$ 
$$ Information \: gain = Entropy(Parent) - Weights average \cdot Entropy(Children) $$

The ID3 algorithm for binary decision trees selects the split with the most information gain possible in every split possible, and does so recursively to arrive at a structure such as

(insert decision tree)

The whole proces is explained step by step in this notebook. First all the helper functions for this algorithm will be introduced, and afterwards these will be combined into the final ID3 algorithm using recursion. 

This is based on the eight part youtube series of Sebastian Mantey on decision trees, which is very helpful if you do not understand certain parts of the algorithm


## data file preparation and importing libraries 

In [1]:
#import libraries needed for both the algorithm and the plot

import pandas as pd             # Will be used for data frames 
import numpy as np              # Will be used for quick matrix algebra as an expansion on pandas
import matplotlib.pyplot as plt # For plotting
import seaborn as sns           # For plotting
import random                   # For randomizing the training and test set selection
from pprint import pprint       # For the printing of the final decision tree
%matplotlib inline               
wine_data = "/Users/Twan/Downloads/winequality-red.csv"

#read csv file path directory
data_frame = pd.read_csv(wine_data, sep = ';', header = True)  
header = data_frame.loc[0,:]
data_frame = data_frame.loc[1::,:]
for col in range(0,12):
    data_frame[col] = pd.to_numeric(data_frame[col], errors='coerce')

data_frame = pd.DataFrame(
    np.row_stack([data_frame.columns, data_frame.values]),
    columns= header)


# Print a small view of the data
data_frame.head()


ImportError: No module named 'seaborn'

First, we make a function which splits the given data randomly in a training set and a test set to test the accuracy later on. In this function, a proportion of the data set is entered to signal the amount for the test size

In [206]:
# array, number --> matrix, matrix
def train_test_split(dataframe, test_proportion):
    """Randomly seperate dataframe into train and test set"""
    
    test_size = round(test_proportion * len(dataframe))
    
    #now we randomize the data set and randomly select test_size amount of indexes 
    total_samples = data_frame.index.tolist()
    test_samples = random.sample(population = total_samples, k = test_size)
    
    #Gather the data of these indexes
    test_data_frame = data_frame.loc[test_samples]
    train_data_frame = data_frame.drop(test_samples)
    
    return train_data_frame, test_data_frame


Below a test for this function

In [207]:
train_data_frame, test_data_frame = train_test_split(data_frame, test_proportion=0.2)

In [208]:
#Now we convert the dataframe to numpy for speed purposes. 

data = train_data_frame.values

# A tiny view
data[0:5]

array([[0.000e+00, 1.000e+00, 2.000e+00, 3.000e+00, 4.000e+00, 5.000e+00,
        6.000e+00, 7.000e+00, 8.000e+00, 9.000e+00, 1.000e+01, 1.100e+01],
       [7.800e+00, 7.600e-01, 4.000e-02, 2.300e+00, 9.200e-02, 1.500e+01,
        5.400e+01, 9.970e-01, 3.260e+00, 6.500e-01, 9.800e+00, 5.000e+00],
       [1.120e+01, 2.800e-01, 5.600e-01, 1.900e+00, 7.500e-02, 1.700e+01,
        6.000e+01, 9.980e-01, 3.160e+00, 5.800e-01, 9.800e+00, 6.000e+00],
       [7.400e+00, 7.000e-01, 0.000e+00, 1.900e+00, 7.600e-02, 1.100e+01,
        3.400e+01, 9.978e-01, 3.510e+00, 5.600e-01, 9.400e+00, 5.000e+00],
       [7.400e+00, 6.600e-01, 0.000e+00, 1.800e+00, 7.500e-02, 1.300e+01,
        4.000e+01, 9.978e-01, 3.510e+00, 5.600e-01, 9.400e+00, 5.000e+00]])

## Some functions

We are first checking if there is only one class available. Although we know this is not the case in this practice file, normally it would be crucial to know. It does this by checking if there is only 1 quality of wine detected. This is important because a tree would be useless if this was the case

In [209]:
# array --> Boolean
def check_if_pure(data):
    """Checks if the data is completely pure"""
    
    
    #The column which you are predicting (the classes)
    quality_column = data[:,-1]
    unique_classes = np.unique(quality_column)
    
    if len(unique_classes) == 1:
        return True
    else:
        return False 


Now we construct a function that checks which class occurs the most in the data set which is input into the function. This is important for the base case of our recursive tree algorithm later on. Because there could occur a situation in which you cannot build your tree any further, and still need to classify. 

In [210]:
# array -> number
def class_classification(data):
    """Checks which classification the majority of given data poses"""
    
    
    quality_column = data[:,-1]
    
    #find the class with the most amount of occurences
    unique_classes, counts_unique_classes = np.unique(quality_column, return_counts = True)
    index = counts_unique_classes.argmax()
    classification = unique_classes[index]
    
    return classification 

This function calculates all potential splits possible in a given category of split (i.e. alcohol percentage, pH value, etc.)

In [212]:
# array --> list
def potential_splits_func(data):
    """Returns dictionary with all potential splits for a given feature"""
    
    
    potential_splits = {}
   
        #extract amount of columns
    _, amount_of_columns = data.shape
    
    #This for-loop extracts the amount of unique values, 
    #to identify the possible splits later
    for column_index in range(0,amount_of_columns -1):
            potential_splits[column_index] = []
        
            values = data[:,column_index]

            unique_values = np.unique(values)
        
            #This calculates the actual split values, it tries and not split at one specific point of the data, but inbetween
            # unique values. 
            for index in range(0,len(unique_values)):
                if index != 0:
                    current_value = unique_values[index]
                    previous_value = unique_values[index -1]
                    potential_split = (current_value + previous_value)/2
                
                    potential_splits[column_index].append(potential_split)
    print(potential_splits)
    return potential_splits


In [196]:
#A plot to help understanding
#Specific example for plot
sns.lmplot(data = train_data_frame, x = 5, y = 8,
           hue = 11, fit_reg = False, aspect = 1.5)

plot.vlines(x=potential_splits[5], ymin =0, ymax =10)

#The same can be done horizontally with plt.hlines!

KeyError: '[ 5  8 11] not in index'

Next up, it is of course important to know after a split which part of the data is split into which category. Since we are dealing with continuous variables in our features, we will call these splits data below threshold and above threshold

In [197]:
# array, number, number --> array, array
def split_data(data, split_column, split_value):
    """Checks which parts are below and above the splitting threshold after the node"""
    
    
    split_column_values = data[:,split_column]
    
    #get data below and above value
    data_below_thr = data[split_column_values <= split_value]
    data_above_thr = data[split_column_values > split_value]
    
    return data_below_thr, data_above_thr

In [198]:
#Specific example for plot
plotting_data_frame = pd.DataFrame(data, columns = data_frame.columns) #CHeck at home!

sns.lmplot(data=plotting_data_frame, x = "pH", y = "alcohol",
          fit_reg = False, size = 6, aspect = 1.5)

plt.vlines(xsplit = split_value, ymin = 0, ymax = 8)

KeyError: "['alcohol' 'pH'] not in index"

The ID3 algorithm tries and split the data for the most favourable entropy level, therefore 
we construct a function which calculates entropy, and one which selects the lowest overall entropy according to the formula of entropy. 

In [199]:
# array --> number
def entropy_func(data):
    """Calculates entropy""" # Remember the formula earlier!
    
    
    quality_column = data[:,-1]
    
    #get counts of classes
    _, counts = np.unique(quality_column, return_counts = True)
    
    probabilities = counts/counts.sum()
    entropy = sum(probabilities * -np.log2(probabilities))
    
    return entropy

In [200]:
# array, array --> number
def split_entropy_func(data_below_thr, data_above_thr):
    """Calculates overall entropy of split"""
    
    
    size = len(data_below_thr) + len(data_above_thr)
    p_data_below_thr = len(data_below_thr) / size
    p_data_above_thr = len(data_above_thr) / size
    
    split_entropy = (p_data_below_thr * entropy_func(data_below_thr)
                       *p_data_above_thr * entropy_func(data_above_thr))
    return split_entropy 


In [201]:
# array, array --> array, array 
def best_split(data, potential_splits):
    """Selects most favourable split based on entropy"""
    
    
    #initialize entropy at arbitrary enormeous value
    split_entropy = 1000
    potential_splits = potential_splits_func(data)
    
    for column_index in potential_splits:
        for value in potential_splits[column_index]:
            data_below_thr, data_above_thr = split_data(data,split_column = column_index, split_value = value)
            current_split_entropy = split_entropy_func(data_below_thr, data_above_thr)
            
            
            # It is not necessary to calculate information gain fully, because the entropy
            # of the parent is always the same in comparing splits!
            if current_split_entropy < split_entropy:
                split_entropy = current_split_entropy
                best_split_column = column_index
                best_split_value = value
            
    
    return best_split_column, best_split_value


Now we are implementing the main algorithm. We do this with recursion where you have a base case and a recursive part of the algorithm to fall back on (Just as you learned in Programming Your World).

In [202]:
# array, (number, number, number) --> dictionary
def decision_tree(data_frame, counter=0, min_samples=2, max_layers=4):
    """Builds the decision tree using previously defined partial functions"""
    
    # data preparations
    if counter == 0:
        data = data_frame.values
        
        # Column headers is defined so that later in the printing of our tree using pprint
        global column_headers
        column_headers = data_frame.columns

    else:
        data = data_frame           
   
    # BASE CASE:
    
    # Checks if data is pure, because then there is no reason to split;
    # Checks if the minimum amount of samples for a split is present;
    # Checks if we did not surpass the predetermined amount of maximum layers in the tree.
    
    if (check_if_pure(data)) or (len(data) < min_samples) or (counter == max_layers):
        
        classification = class_classification(data)
        return classification

    
    # RECURSIVE PART
    else:    
        # Counter for recursion
        counter += 1

        # Extract best split from potential splits, and determine the new classifications
        
        potential_splits = potential_splits_func(data)
        split_column, split_value = best_split(data, potential_splits)
        data_below_thr, data_above_thr = split_data(data, split_column, split_value)
        
        # Reapply the function to these new categories
        below_answer = decision_tree(data_below_thr, counter, min_samples, max_layers)
        above_answer = decision_tree(data_above_thr, counter, min_samples, max_layers)
        
        # Printing the tree
        # The name of the feature on which is being split
        feature_name = column_headers[split_column]
        
        # This is the split question. It is of the following form
        "Is {insert variable on which is split} below or equal to {insert value}?"
        question = "{} <= {}?".format(feature_name, split_value)
        
        branch = {question : []}
        
        # If a split yields two categories with the same categorisation, it is useless. 
        if below_answer == above_answer:
            branch = below_answer
            
            # Else of course not
        else:
            branch[question].append(below_answer)
            branch[question].append(above_answer)
        
        
        return branch
        

Notice that this function cannot make splits along categories, how would you implement this?

## Test phase

Now our code is finished and we can finally build our decision tree! However, we cannot yet classify new examples, and we need to be able to do this so that we can test our decision tree and without this function, it would also not have any use. So let's build this final step. Just like the building of the tree, this is also a recursive algorithm 

In [203]:
#array, dictionary --> dictionary
def classify(example, tree):
    """Follows along a decision tree to classify a new datapoint"""
    
    # Extracts the split condition
    question = list(tree.keys())[0]
    feature_name, comparison_operator, value = question.split(" ")
    
    # Choose path according to this comparison
    
    if example[feature_name] <= float(value):
        answer = tree[question][0]
    else:
        answer = tree[question][1]
        
        
    #BASE CASE: Why does this work?
    if not isinstance(answer, dict):
        return answer
    
    # RECURSIVE PART:
    else:
        tree = answer 
        return classify(example, tree)
    

In [213]:
wine_tree = decision_tree(train_data_frame)

pprint(wine_tree)

{0: [2.3, 4.75, 4.95, 5.05, 5.15, 5.25, 5.35, 5.45, 5.55, 5.65, 5.75, 5.85, 5.95, 6.05, 6.15, 6.25, 6.35, 6.45, 6.55, 6.65, 6.75, 6.85, 6.95, 7.05, 7.15, 7.25, 7.35, 7.45, 7.55, 7.65, 7.75, 7.85, 7.95, 8.05, 8.149999999999999, 8.25, 8.350000000000001, 8.45, 8.55, 8.649999999999999, 8.75, 8.850000000000001, 8.95, 9.05, 9.149999999999999, 9.25, 9.350000000000001, 9.45, 9.55, 9.649999999999999, 9.75, 9.850000000000001, 9.95, 10.05, 10.149999999999999, 10.25, 10.350000000000001, 10.45, 10.55, 10.649999999999999, 10.75, 10.850000000000001, 10.95, 11.05, 11.149999999999999, 11.25, 11.350000000000001, 11.45, 11.55, 11.649999999999999, 11.75, 11.850000000000001, 11.95, 12.05, 12.149999999999999, 12.25, 12.350000000000001, 12.45, 12.55, 12.649999999999999, 12.75, 12.850000000000001, 12.95, 13.1, 13.25, 13.350000000000001, 13.45, 13.6, 14.0, 14.65, 15.25, 15.55], 1: [0.14, 0.16999999999999998, 0.185, 0.195, 0.20500000000000002, 0.215, 0.225, 0.235, 0.245, 0.255, 0.265, 0.275, 0.28500000000000003

## Accuracy

Now We test the accuracy of our tree!

In [214]:
#array, dictionary --> number
def calculate_accuracy(data_frame, tree):
    """Calculates accuracy of algorithm"""
    
    #Classification
    data_frame["classification"] = data_frame.apply(classify, axis=1, args=(tree,))
    
    #Check if classification is correct 
    data_frame["classification_correct"] = data_frame["classification"] == data_frame["quality"]
    
    accuracy = data_frame["classification_correct"].mean()
    
    return accuracy 

calculate_accuracy(test_data_frame, wine_tree)


ValueError: ('too many values to unpack (expected 3)', 'occurred at index 280')